In [1]:
#######################################################################
############### ATMOSPHERIC CORRECTION USING PY6S #####################
#######################################################################
## Daniel Borini Alves; Dhemerson Conciani; and Swanni Alvarado

## --  adapted  based   on  Minh Nguyen  script   <https://github.com/ndminhhus/geeguide/blob/master/02.Atm-correction.md>  and 
## the original script developed by Sam Murphy <https://github.com/samsammurphy/gee-atmcorr-S2>, using PY6s python API v1.8.0,
## <https://py6s.readthedocs.io/en/latest/> (Robin Wilson). See all references in the end of the script.

## Inputs: Top of Atmosphere (TOA) reflectance images (Sentinel 2 and Landsat collections)
## Output: Bottom of Atmosphere (BOA) reflectance of each collection, exported in Google Drive

## ACTIVATE PACKAGES
from Py6S import * 
import datetime
import math
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
from atmospheric import Atmospheric # custom-defined class by Sam Murphy -- check link above
import ee

ee.Initialize()

In [2]:
## FILTER IMAGE COLLECTIONS BY SENSORS/PLATFORMS

# Define your time interval, centroid of study area and area of interest
startDate = ee.Date('2019-08-01')
endDate   = ee.Date('2019-10-01') #long periods can spend a lot of processing time...
geom = ee.Geometry.Point(-61.5775286643,-8.4852883834)
#region = geom.buffer(20000).bounds().getInfo()['coordinates']
region = ee.FeatureCollection("users/shps/Recorte_ZonasAeB_Sent2")

print ('Available images in the period:')
# Sentinel 2A collection images
S2A_col = ee.ImageCollection('COPERNICUS/S2').filterBounds(geom).filterDate(startDate,endDate).filterMetadata('SPACECRAFT_NAME', 'equals', 'Sentinel-2A')
col_length_S2A = S2A_col.size().getInfo()
print ('S2A images = ', col_length_S2A)

# Sentinel 2B collection images
S2B_col = ee.ImageCollection('COPERNICUS/S2').filterBounds(geom).filterDate(startDate,endDate).filterMetadata('SPACECRAFT_NAME', 'equals', 'Sentinel-2B')
col_length_S2B = S2B_col.size().getInfo()
print ('S2B images = ', col_length_S2B)

# Landsat OLI collection images
L8_col = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterBounds(geom).filterDate(startDate,endDate)
col_length_L8 = L8_col.size().getInfo()
print ('L8 images = ', col_length_L8)

# Landsat ETM+ collection images
L7_col = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA').filterBounds(geom).filterDate(startDate,endDate)
col_length_L7 = L7_col.size().getInfo()
print ('L7 images = ', col_length_L7)

Available images in the period:
S2A images =  5
S2B images =  6
L8 images =  3
L7 images =  4


In [24]:
### SENTINEL 2A COLLECTION ATMOSPHERIC CORRECTION

print('Sentinel 2A atmopheric correction')

for i in range (0,col_length_S2A):
    # OPEN IMAGE AND EXTRACT METADATA ATMOSPHERIC PARAMETERS
    
    S2_list = S2A_col.toList(col_length_S2A)
    img = ee.Image(S2_list.get(i))
    
    info = img.getInfo()['properties']
    scene_date = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000)
    solar_z = img.getInfo()['properties']['MEAN_SOLAR_ZENITH_ANGLE']
    h2o = Atmospheric.water(geom,img.date()).getInfo()
    o3 = Atmospheric.ozone(geom,img.date()).getInfo()
    aot = Atmospheric.aerosol(geom,img.date()).getInfo()
    SRTM = ee.Image('CGIAR/SRTM90_V4')  # Shuttle Radar Topography mission covers *most* of the Earth
    alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = geom.centroid()).get('elevation').getInfo()
    km = alt/1000                       # i.e. Py6S uses units of kilometers
    
    # APPLY CORRECTION
    
    s = SixS()
    # Atmospheric constituents
    s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
    s.aero_profile = AtmosProfile.PredefinedType(AtmosProfile.Tropical) #altered from- s.aero_profile = AeroProfile.Continental #
    s.aot550 = aot
    # Earth-Sun-satellite geometry
    s.geometry = Geometry.User()
    s.geometry.view_z = 0               # always NADIR (I think..)
    s.geometry.solar_z = solar_z        # solar zenith angle
    s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
    s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
    s.altitudes.set_sensor_satellite_level()
    s.altitudes.set_target_custom_altitude(km)
    
    def spectralResponseFunction(bandname):        
        
        bandSelect = {
            'B1':PredefinedWavelengths.S2A_MSI_01,
            'B2':PredefinedWavelengths.S2A_MSI_02,
            'B3':PredefinedWavelengths.S2A_MSI_03,
            'B4':PredefinedWavelengths.S2A_MSI_04,
            'B5':PredefinedWavelengths.S2A_MSI_05,
            'B6':PredefinedWavelengths.S2A_MSI_06,
            'B7':PredefinedWavelengths.S2A_MSI_07,
            'B8':PredefinedWavelengths.S2A_MSI_08,
            'B8A':PredefinedWavelengths.S2A_MSI_8A,
            'B9':PredefinedWavelengths.S2A_MSI_09,
            'B10':PredefinedWavelengths.S2A_MSI_10,
            'B11':PredefinedWavelengths.S2A_MSI_11,
            'B12':PredefinedWavelengths.S2A_MSI_12,
                        }
        return Wavelength(bandSelect[bandname])
    def toa_to_rad(bandname):
        ESUN = info['SOLAR_IRRADIANCE_'+bandname]
        solar_angle_correction = math.cos(math.radians(solar_z))
        # Earth-Sun distance (from day of year)
        doy = scene_date.timetuple().tm_yday
        d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
        # conversion factor
        multiplier = ESUN*solar_angle_correction/(math.pi*d**2)
        # at-sensor radiance
        rad = img.select(bandname).multiply(multiplier)
        return rad
    def surface_reflectance(bandname):
        # run 6S for this waveband
        s.wavelength = spectralResponseFunction(bandname)
        s.run()

        # extract 6S outputs
        Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
        Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
        Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
        absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
        scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
        tau2 = absorb*scatter                                #total transmissivity

        # radiance to surface reflectance
        rad = toa_to_rad(bandname)
        ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
        return ref
    
    #dblue  = surface_reflectance('B1') #costal aerosol 
    blue = surface_reflectance('B2')
    green = surface_reflectance('B3')
    red = surface_reflectance('B4')
    #redge1 = surface_reflectance('B5')
    #redge2 = surface_reflectance('B6')
    #redge3 = surface_reflectance('B7')
    #nir0 = surface_reflectance('B8')
    nir = surface_reflectance('B8A') # used as NIR due to better compatibility with Landsat <doi.org/10.1016/j.rse.2018.04.031>
    #wvap = surface_reflectance('B9')
    #cirrus = surface_reflectance('B10')
    swir1 = surface_reflectance('B11')
    swir2 = surface_reflectance('B12')
    #qa = img.select('QA60')
    
    # to use all bands (-- activate alsdo the bands in the list above)
    #boa = dblue.addBands(blue).addBands(green).addBands(red).addBands(redge1).addBands(redge2).addBands(redge3).addBands(nir0).addBands(nir).addBands(wvap).addBands(cirrus).addBands(swir1).addBands(swir2) #.addBands(qa)
    # selected bands
    boa = blue.addBands(green).addBands(red).addBands(nir).addBands(swir1).addBands(swir2)
    
    # EXPORT FINAL IMAGE
    
    #fname = img.get('DATASTRIP_ID').getInfo()
    #fname = str(fname [25:33]) + '_' + str(fname [0:3])
    fname = ee.String(img.get('GRANULE_ID')).getInfo()
    task = ee.batch.Export.image.toDrive(**{
        'image': boa.unmask(-9999),
        'description': fname + '_BOA',
        'folder':'P6s_export',
        'scale': 20,
        'fileFormat': 'GeoTIFF',
        'region': region.geometry().bounds(),
        'maxPixels': 1e13
    })

    task.start()
    print('exporting ' + fname + '--->done')

Sentinel 2A atmopheric correction
exporting L1C_T20LPR_A021500_20190804T143101--->done
exporting L1C_T20LPR_A021643_20190814T142756--->done
exporting L1C_T20LPR_A021786_20190824T142754--->done
exporting L1C_T20LPR_A021929_20190903T142959--->done
exporting L1C_T20LPR_A022072_20190913T142751--->done


In [25]:
### SENTINEL 2B COLLECTION ATMOSPHERIC CORRECTION

print('Sentinel 2B atmopheric correction')

for i in range (0,col_length_S2B):
    # OPEN IMAGE AND EXTRACT METADATA ATMOSPHERIC PARAMETERS
    
    S2_list = S2B_col.toList(col_length_S2B)
    img = ee.Image(S2_list.get(i))
    
    info = img.getInfo()['properties']
    scene_date = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000)
    solar_z = img.getInfo()['properties']['MEAN_SOLAR_ZENITH_ANGLE']
    h2o = Atmospheric.water(geom,img.date()).getInfo()
    o3 = Atmospheric.ozone(geom,img.date()).getInfo()
    aot = Atmospheric.aerosol(geom,img.date()).getInfo()
    SRTM = ee.Image('CGIAR/SRTM90_V4')  # Shuttle Radar Topography mission covers *most* of the Earth
    alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = geom.centroid()).get('elevation').getInfo()
    km = alt/1000                       # i.e. Py6S uses units of kilometers
    
    # APPLY CORRECTION
    
    s = SixS()
    # Atmospheric constituents
    s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
    s.aero_profile = AtmosProfile.PredefinedType(AtmosProfile.Tropical)  #altered from-s.aero_profile = AeroProfile.Continental
    s.aot550 = aot
    # Earth-Sun-satellite geometry
    s.geometry = Geometry.User()
    s.geometry.view_z = 0               # always NADIR (I think..)
    s.geometry.solar_z = solar_z        # solar zenith angle
    s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
    s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
    s.altitudes.set_sensor_satellite_level()
    s.altitudes.set_target_custom_altitude(km)
    
    def spectralResponseFunction(bandname):        
        
        bandSelect = {
            'B1':PredefinedWavelengths.S2B_MSI_01,
            'B2':PredefinedWavelengths.S2B_MSI_02,
            'B3':PredefinedWavelengths.S2B_MSI_03,
            'B4':PredefinedWavelengths.S2B_MSI_04,
            'B5':PredefinedWavelengths.S2B_MSI_05,
            'B6':PredefinedWavelengths.S2B_MSI_06,
            'B7':PredefinedWavelengths.S2B_MSI_07,
            'B8':PredefinedWavelengths.S2B_MSI_08,
            'B8A':PredefinedWavelengths.S2B_MSI_8A,
            'B9':PredefinedWavelengths.S2B_MSI_09,
            'B10':PredefinedWavelengths.S2B_MSI_10,
            'B11':PredefinedWavelengths.S2B_MSI_11,
            'B12':PredefinedWavelengths.S2B_MSI_12,
                        }
        return Wavelength(bandSelect[bandname])
    def toa_to_rad(bandname):
        ESUN = info['SOLAR_IRRADIANCE_'+bandname]
        solar_angle_correction = math.cos(math.radians(solar_z))
        # Earth-Sun distance (from day of year)
        doy = scene_date.timetuple().tm_yday
        d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
        # conversion factor
        multiplier = ESUN*solar_angle_correction/(math.pi*d**2)
        # at-sensor radiance
        rad = img.select(bandname).multiply(multiplier)
        return rad
    def surface_reflectance(bandname):
        # run 6S for this waveband
        s.wavelength = spectralResponseFunction(bandname)
        s.run()

        # extract 6S outputs
        Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
        Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
        Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
        absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
        scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
        tau2 = absorb*scatter                                #total transmissivity

        # radiance to surface reflectance
        rad = toa_to_rad(bandname)
        ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
        return ref
    
    #dblue  = surface_reflectance('B1') #costal aerosol 
    blue = surface_reflectance('B2')
    green = surface_reflectance('B3')
    red = surface_reflectance('B4')
    #redge1 = surface_reflectance('B5')
    #redge2 = surface_reflectance('B6')
    #redge3 = surface_reflectance('B7')
    #nir0 = surface_reflectance('B8')
    nir = surface_reflectance('B8A') # used as NIR due to better compatibility with Landsat <doi.org/10.1016/j.rse.2018.04.031>
    #wvap = surface_reflectance('B9')
    #cirrus = surface_reflectance('B10')
    swir1 = surface_reflectance('B11')
    swir2 = surface_reflectance('B12')
    #qa = img.select('QA60')
    
    # to use all bands (-- activate alsdo the bands in the list above)
    #boa = dblue.addBands(blue).addBands(green).addBands(red).addBands(redge1).addBands(redge2).addBands(redge3).addBands(nir0).addBands(nir).addBands(wvap).addBands(cirrus).addBands(swir1).addBands(swir2) #.addBands(qa)
    # selected bands
    boa = blue.addBands(green).addBands(red).addBands(nir).addBands(swir1).addBands(swir2)
    
    # EXPORT FINAL IMAGE
    
    #fname = img.get('DATASTRIP_ID').getInfo()
    #fname = str(fname [25:33]) + '_' + str(fname [0:3])
    fname = ee.String(img.get('GRANULE_ID')).getInfo() ### adicionado
    task = ee.batch.Export.image.toDrive(**{
        'image': boa.unmask(-9999),
        'description': fname + '_BOA',
        'folder':'P6s_export',
        'scale': 20,
        'fileFormat': 'GeoTIFF',
        'region': region.geometry().bounds(),
        'maxPixels': 1e13
    })

    task.start()
    print('exporting ' + fname + '--->done')

Sentinel 2B atmopheric correction
exporting L1C_T20LPR_A012663_20190809T142759--->done
exporting L1C_T20LPR_A012806_20190819T142758--->done
exporting L1C_T20LPR_A012949_20190829T142756--->done
exporting L1C_T20LPR_A013092_20190908T142754--->done
exporting L1C_T20LPR_A013235_20190918T143240--->done
exporting L1C_T20LPR_A013378_20190928T142753--->done


In [26]:
### LANDSAT 8 COLLECTION ATMOSPHERIC CORRECTION

print('Landsat 8 atmopheric correction')

for i in range (0,col_length_L8):
    # OPEN IMAGE AND EXTRACT METADATA ATMOSPHERIC PARAMETERS
    
    L8_list = L8_col.toList(col_length_L8)
    img = ee.Image(L8_list.get(i))

    info = img.getInfo()['properties']
    scene_date = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000)
    solar_elv = img.getInfo()['properties']['SUN_ELEVATION']
    solar_z = ee.Number(90).subtract(solar_elv).getInfo()
    h2o = Atmospheric.water(geom,img.date()).getInfo()
    o3 = Atmospheric.ozone(geom,img.date()).getInfo()
    aot = Atmospheric.aerosol(geom,img.date()).getInfo()
    SRTM = ee.Image('CGIAR/SRTM90_V4') # Shuttle Radar Topography mission covers *most* of the Earth
    alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = geom.centroid()).get('elevation').getInfo()
    km = alt/1000                      # i.e. Py6S uses units of kilometers
    
    # APPLY CORRECTION
    
    s = SixS()
    # Atmospheric constituents
    s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
    s.aero_profile = AtmosProfile.PredefinedType(AtmosProfile.Tropical) #altered from- s.aero_profile = AeroProfile.Continental
    s.aot550 = aot
    # Earth-Sun-satellite geometry
    s.geometry = Geometry.User()
    s.geometry.view_z = 0               # always NADIR (I think..)
    s.geometry.solar_z = solar_z        # solar zenith angle
    s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
    s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
    s.altitudes.set_sensor_satellite_level()
    s.altitudes.set_target_custom_altitude(km)
    
    def spectralResponseFunction(bandname):        
        bandSelect = {
            'B1':PredefinedWavelengths.LANDSAT_OLI_B1,
            'B2':PredefinedWavelengths.LANDSAT_OLI_B2,
            'B3':PredefinedWavelengths.LANDSAT_OLI_B3,
            'B4':PredefinedWavelengths.LANDSAT_OLI_B4,
            'B5':PredefinedWavelengths.LANDSAT_OLI_B5,
            'B6':PredefinedWavelengths.LANDSAT_OLI_B6,
            'B7':PredefinedWavelengths.LANDSAT_OLI_B7,
            'B8':PredefinedWavelengths.LANDSAT_OLI_B8,
            'B9':PredefinedWavelengths.LANDSAT_OLI_B9,
                    }
        return Wavelength(bandSelect[bandname])
    def toa_to_rad(bandname):
        ESUN_L8 = [1895.33, 2004.57, 1820.75, 1549.49, 951.76, 247.55, 85.46, 1723.8, 366.97]
        ESUN_BAND = {
            'B1':ESUN_L8[0],
            'B2':ESUN_L8[1],
            'B3':ESUN_L8[2],
            'B4':ESUN_L8[3],
            'B5':ESUN_L8[4],
            'B6':ESUN_L8[5],
            'B7':ESUN_L8[6],
            'B8':ESUN_L8[7],
            'B9':ESUN_L8[8],
            }
        solar_angle_correction = math.cos(math.radians(solar_z))
        # Earth-Sun distance (from day of year)
        doy = scene_date.timetuple().tm_yday
        d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
        # conversion factor
        multiplier = ESUN_BAND[bandname]*solar_angle_correction/(math.pi*d**2)
        # at-sensor radiance
        rad = img.select(bandname).multiply(multiplier)
        return rad
    def surface_reflectance(bandname):
        # run 6S for this waveband
        s.wavelength = spectralResponseFunction(bandname)
        s.run()

        # extract 6S outputs
        Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
        Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
        Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
        absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
        scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
        tau2 = absorb*scatter                                #total transmissivity

        # radiance to surface reflectance
        rad = toa_to_rad(bandname)
        ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
        return ref
    
    #dblue  = surface_reflectance('B1') #costal aerosol 
    blue = surface_reflectance('B2')
    green = surface_reflectance('B3')
    red = surface_reflectance('B4')
    nir = surface_reflectance('B5')
    swir1 = surface_reflectance('B6')
    swir2 = surface_reflectance('B7')
    #pan = surface_reflectance('B8')
    #cirrus = surface_reflectance('B9')
    #qa = img.select('BQA')
    
    # to use all bands (-- activate alsdo the bands in the list above)
    #boa = dblue.addBands(blue).addBands(green).addBands(red).addBands(nir).addBands(swir1).addBands(swir2) #.addBands(qa)
    # selected bands
    boa = blue.addBands(green).addBands(red).addBands(nir).addBands(swir1).addBands(swir2)
    
    # EXPORT FINAL IMAGE
    
    fname = ee.String(img.get('LANDSAT_PRODUCT_ID')).getInfo()
    #fname = str(fname [12:20]) + '_L08'

    task = ee.batch.Export.image.toDrive(**{
        'image': boa.unmask(-9999),
        'description': fname + '_BOA',
        'folder':'P6s_export',
        'scale': 30,
        'fileFormat': 'GeoTIFF',
        'region': region.geometry().bounds(),
        'maxPixels': 1e13
    })

    task.start()
    print('exporting ' + fname + '--->done')

Landsat 8 atmopheric correction
exporting LC08_L1TP_231066_20190805_20190820_01_T1--->done
exporting LC08_L1TP_231066_20190821_20190903_01_T1--->done
exporting LC08_L1TP_231066_20190906_20190917_01_T1--->done


In [3]:
### LANDSAT 7 COLLECTION ATMOSPHERIC CORRECTION

print('Landsat 7 atmopheric correction')

for i in range (0,col_length_L7):
    # OPEN IMAGE AND EXTRACT METADATA ATMOSPHERIC PARAMETERS
    
    L7_list = L7_col.toList(col_length_L7)
    img = ee.Image(L7_list.get(i))

    info = img.getInfo()['properties']
    scene_date = datetime.datetime.utcfromtimestamp(info['system:time_start']/1000)
    solar_elv = img.getInfo()['properties']['SUN_ELEVATION']
    solar_z = ee.Number(90).subtract(solar_elv).getInfo()
    h2o = Atmospheric.water(geom,img.date()).getInfo()
    o3 = Atmospheric.ozone(geom,img.date()).getInfo()
    aot = Atmospheric.aerosol(geom,img.date()).getInfo()
    SRTM = ee.Image('CGIAR/SRTM90_V4')  # Shuttle Radar Topography mission covers *most* of the Earth
    alt = SRTM.reduceRegion(reducer = ee.Reducer.mean(),geometry = geom.centroid()).get('elevation').getInfo()
    km = alt/1000                       # i.e. Py6S uses units of kilometers
    
    # APPLY CORRECTION
    
    s = SixS()
    # Atmospheric constituents
    s.atmos_profile = AtmosProfile.UserWaterAndOzone(h2o,o3)
    s.aero_profile = AeroProfile.Continental # altered from: s.aero_profile = AtmosProfile.PredefinedType(AtmosProfile.Tropical)
    s.aot550 = aot
    # Earth-Sun-satellite geometry
    s.geometry = Geometry.User()
    s.geometry.view_z = 0               # always NADIR (I think..)
    s.geometry.solar_z = solar_z        # solar zenith angle
    s.geometry.month = scene_date.month # month and day used for Earth-Sun distance
    s.geometry.day = scene_date.day     # month and day used for Earth-Sun distance
    s.altitudes.set_sensor_satellite_level()
    s.altitudes.set_target_custom_altitude(km)
    
    def spectralResponseFunction(bandname):        
        bandSelect = {
            'B1':PredefinedWavelengths.LANDSAT_ETM_B1,
            'B2':PredefinedWavelengths.LANDSAT_ETM_B2,
            'B3':PredefinedWavelengths.LANDSAT_ETM_B3,
            'B4':PredefinedWavelengths.LANDSAT_ETM_B4,
            'B5':PredefinedWavelengths.LANDSAT_ETM_B5,
            'B7':PredefinedWavelengths.LANDSAT_ETM_B7,
                    }
        return Wavelength(bandSelect[bandname])
    def toa_to_rad(bandname):
        ESUN_L8 = [1895.33, 2004.57, 1820.75, 1549.49, 951.76, 247.55, 85.46, 1723.8, 366.97]
        ESUN_L7 = [1997, 1812, 1533, 1039, 230.8, 84.9] # PAN =  1362 (removed to match Py6S)
        ESUN_BAND = {
            'B1':ESUN_L7[0],
            'B2':ESUN_L7[1],
            'B3':ESUN_L7[2],
            'B4':ESUN_L7[3],
            'B5':ESUN_L7[4],            
            'B7':ESUN_L7[5],
           }
        solar_angle_correction = math.cos(math.radians(solar_z))
        # Earth-Sun distance (from day of year)
        doy = scene_date.timetuple().tm_yday
        d = 1 - 0.01672 * math.cos(0.9856 * (doy-4))# http://physics.stackexchange.com/questions/177949/earth-sun-distance-on-a-given-day-of-the-year
        # conversion factor
        multiplier = ESUN_BAND[bandname]*solar_angle_correction/(math.pi*d**2)
        # at-sensor radiance
        rad = img.select(bandname).multiply(multiplier)
        return rad
    def surface_reflectance(bandname):
        # run 6S for this waveband
        s.wavelength = spectralResponseFunction(bandname)
        s.run()

        # extract 6S outputs
        Edir = s.outputs.direct_solar_irradiance             #direct solar irradiance
        Edif = s.outputs.diffuse_solar_irradiance            #diffuse solar irradiance
        Lp   = s.outputs.atmospheric_intrinsic_radiance      #path radiance
        absorb  = s.outputs.trans['global_gas'].upward       #absorption transmissivity
        scatter = s.outputs.trans['total_scattering'].upward #scattering transmissivity
        tau2 = absorb*scatter                                #total transmissivity

        # radiance to surface reflectance
        rad = toa_to_rad(bandname)
        ref = rad.subtract(Lp).multiply(math.pi).divide(tau2*(Edir+Edif))
        return ref
    
    blue = surface_reflectance('B1')
    green = surface_reflectance('B2')
    red = surface_reflectance('B3')
    nir = surface_reflectance('B4')    
    swir1 = surface_reflectance('B5')
    swir2 = surface_reflectance('B7')
        
    boa = blue.addBands(green).addBands(red).addBands(nir).addBands(swir1).addBands(swir2) ###
        
    
    # EXPORT FINAL IMAGE
    fname = ee.String(img.get('LANDSAT_PRODUCT_ID')).getInfo()
    #fname = str(fname [12:20]) + '_L07'
    
    task = ee.batch.Export.image.toDrive(**{
        'image': boa.unmask(-9999),
        'description': fname + '_BOA',
        'folder':'P6s_export',
        'scale': 30,
        'fileFormat': 'GeoTIFF',
        'region': region.geometry().bounds(),
        'maxPixels': 1e13
    })

    task.start()
    print('exporting ' + fname + '--->done')

Landsat 7 atmopheric correction
exporting LE07_L1TP_231066_20190813_20190908_01_T1--->done
exporting LE07_L1TP_231066_20190829_20190924_01_T1--->done
exporting LE07_L1TP_231066_20190914_20191028_01_T1--->done
exporting LE07_L1TP_231066_20190930_20191026_01_T1--->done


In [ ]:
## REFERENCES ##

# -- PY6s Python API
# WILSON, R. T., Py6S: A Python interface to the 6S radiative transfer model, Computers and Geosciences, 51, p.166-171, 2013.

# -- Second Simulation of the Satellite Signal in the Solar Spectrum (6S) method 
# VERMOTE, E. F.; TANRÉ, D.; DEUZÉ, J. L.; HERMAN, M.; MORCRETTE, J.-J. Second simulation of the satellite signal in the solar spectrum, 6S:
# an overview. IEEE Transactions on Geoscience and Remote Sensing, v. 35, n. 3, p. 675–686, 1997. 
# VERMOTE, E. F.; EL SALEOUS, N.; JUSTICE, C. O.; KAUFMAN, Y. J.; PRIVETTE, J. L.; REMER, L.; ROGER, J. C.; TANRÉ, D. Atmospheric 
# correction of visible to middle-infrared EOS-MODIS data over land surfaces: Background, operational algorithm and validation. Journal 
# of Geophysical Research, v. 102, p. 17131, 1997. 

# -- Harmonization of Landsat-Sentinel2 example, in the context of crop monitoring (GEE scripts available, used as reference here)
# NGUYEN, M. D.; BAEZ-VILLANUEVA, O. M.; BUI, D. D.; NGUYEN, P. T.; RIBBE, L. Harmonization of Landsat and Sentinel 2 for crop monitoring 
# in drought prone areas: Case studies of Ninh Thuan (Vietnam) and Bekaa (Lebanon). Remote Sensing, v. 12, n. 2, p. 1–18, 2020.